In [1]:
from transformers.trainer_utils import set_seed

SEED = 6
set_seed(SEED)

## Initialising the same transformer as in the other notebook


In [2]:
from transformers import AutoConfig, LlamaConfig


# Llama_config = LlamaConfig.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0", num_hidden_layers = 1, use_cache = False, hidden_size = 4, num_attention_heads = 1, 
#                                            output_hidden_states=True,  num_key_value_heads = 1, past_key_values = True)


Llama_config = LlamaConfig.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0", num_hidden_layers = 1, use_cache = False, hidden_size = 8, num_attention_heads = 4, 
                                           output_hidden_states=True,  num_key_value_heads = 2, past_key_values = True)


Llama_config

LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 8,
  "initializer_range": 0.02,
  "intermediate_size": 5632,
  "max_position_embeddings": 2048,
  "model_type": "llama",
  "num_attention_heads": 4,
  "num_hidden_layers": 1,
  "num_key_value_heads": 2,
  "output_hidden_states": true,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.38.0.dev0",
  "use_cache": false,
  "vocab_size": 32000
}

In [3]:
from transformers import AutoModel

tinyllama = AutoModel.from_config(Llama_config)

In [4]:
from transformers import LlamaTokenizer

src_sent = "hi how are"

llama_tokenizer = LlamaTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

In [5]:
tokenized_src_dict = llama_tokenizer.encode_plus(src_sent, return_tensors='pt')
tokenized_src_dict

{'input_ids': tensor([[   1, 7251,  920,  526]]), 'attention_mask': tensor([[1, 1, 1, 1]])}

In [6]:
src_tokenized = tokenized_src_dict["input_ids"]
src_tokenized

tensor([[   1, 7251,  920,  526]])

In [7]:
llama_tokenizer.decode(*src_tokenized)

'<s> hi how are'

## Manual computation of 1 layer of llama block

## Defining the hyperparameters

In [8]:
embed_dim = d_model = 8

num_heads = 4

n_heads_q = num_heads

n_kv_heads = 2

seq_len = 4

head_dim = embed_dim // num_heads

n_rep = n_heads_q//n_kv_heads
print("n_rep = ", n_rep)

n_rep =  2


In [9]:
import numpy as np

src_tokenized_np = src_tokenized.numpy()[0]
src_tokenized_np.shape

(4,)

In [10]:
state_dict = tinyllama.state_dict()

### 1. Source token embeddings 

In [11]:
import numpy as np


src_vocab_embeds = state_dict["embed_tokens.weight"].numpy()

src_embedding = np.zeros((src_tokenized_np.shape[0], d_model))

for i in range(src_tokenized_np.shape[0]):
        word_index = src_tokenized_np[i]
        if word_index < 0 or word_index >= src_vocab_embeds.shape[0]:
            raise ValueError(f"Invalid word index: {word_index}")
        src_embedding[i, :] = src_vocab_embeds[word_index, :]

        print(f"Word index: {word_index}, Embedding: {src_vocab_embeds[word_index, :]}")
print()
print(src_embedding.shape)


Word index: 1, Embedding: [-0.02582919  0.02102008 -0.04830853 -0.00791116 -0.01639724 -0.01398921
 -0.01635511 -0.00059234]
Word index: 7251, Embedding: [ 0.00742205 -0.03997844  0.03038153 -0.01515712  0.01529675  0.02885903
  0.00369752  0.02495434]
Word index: 920, Embedding: [ 0.0290757   0.00363495 -0.01033172 -0.01439318  0.00574241 -0.00303749
 -0.01590108 -0.04089781]
Word index: 526, Embedding: [ 0.02144257 -0.01138141 -0.00028663  0.04138773  0.0012412   0.00799416
 -0.02266162  0.01042366]

(4, 8)


In [12]:
residual = src_embedding

### 2. Pre-normalization (RMSNorm)

In [13]:
variance_epsilon = 1e-05

wt = state_dict["layers.0.input_layernorm.weight"].numpy()

dtype = src_embedding.dtype
src_embedding = src_embedding.astype(np.float32)
variance = np.mean(src_embedding**2, axis=-1, keepdims=True)
src_embedding = src_embedding * (1/np.sqrt(variance + variance_epsilon))

hidden_state = wt * src_embedding

In [14]:
hidden_state, hidden_state.shape

(array([[-1.1118075 ,  0.9048014 , -2.0794218 , -0.3405329 , -0.705813  ,
         -0.60216004, -0.7039994 , -0.02549705],
        [ 0.3098795 , -1.669147  ,  1.2684647 , -0.6328275 ,  0.6386575 ,
          1.2048986 ,  0.15437567,  1.0418732 ],
        [ 1.4492195 ,  0.18117674, -0.51496375, -0.71739876,  0.28621894,
         -0.15139766, -0.7925571 , -2.0384686 ],
        [ 1.0945936 , -0.58099484, -0.01463168,  2.1127481 ,  0.06336023,
          0.40808326, -1.1568234 ,  0.53210396]], dtype=float32),
 (4, 8))

### 3. Getting the Q,K and V matrices for attention calcualtion

In [16]:
Wq = state_dict["layers.0.self_attn.q_proj.weight"].numpy()
Wk = state_dict["layers.0.self_attn.k_proj.weight"].numpy()
Wv = state_dict["layers.0.self_attn.v_proj.weight"].numpy()


query = np.matmul(hidden_state, Wq.T)
key = np.matmul(hidden_state, Wk.T)
value = np.matmul(hidden_state, Wv.T)

In [17]:
query, key, value

(array([[ 0.050677  ,  0.01442977,  0.07178964, -0.00791767, -0.05778811,
          0.01414968, -0.11290798,  0.06666089],
        [-0.01080457,  0.05896645, -0.05865115,  0.09133947,  0.03257699,
         -0.06912354,  0.00179233, -0.02657457],
        [-0.07446965,  0.00438682,  0.02495936, -0.13715227,  0.08285801,
          0.08471757,  0.09645503, -0.13390933],
        [-0.01554403, -0.02120098, -0.03709128,  0.04432562, -0.03873054,
         -0.08261885,  0.05573605,  0.03836229]], dtype=float32),
 array([[-0.07981046, -0.02992112, -0.0730254 , -0.07158848],
        [ 0.06769638,  0.02315141,  0.0885109 ,  0.06340569],
        [-0.00255612, -0.02511003, -0.03655827,  0.07421161],
        [-0.04312804, -0.02189272,  0.00207526, -0.03560939]],
       dtype=float32),
 array([[ 0.00158199, -0.06093168, -0.03399701, -0.04116755],
        [ 0.02432493,  0.04214872, -0.01122927, -0.00305679],
        [-0.05153289,  0.02271163, -0.0062439 ,  0.0635836 ],
        [ 0.01803326,  0.00495319

In [18]:
print("query_shape = ", query.shape)
print("key_shape = ", key.shape)
print("value_shape = ", value.shape)
print()

query_shape =  (4, 8)
key_shape =  (4, 4)
value_shape =  (4, 4)



In [19]:
q_len, _ = query.shape


print("After reshaping... \n")

query1 = np.transpose(np.reshape(query, (q_len, num_heads, head_dim)), (1, 0, 2))
key1 = np.transpose(np.reshape(key, (q_len, n_kv_heads, head_dim)), (1, 0, 2))
value1 = np.transpose(np.reshape(value, (q_len, n_kv_heads, head_dim)), (1, 0, 2))

print("query_shape = ", query1.shape)
print("key_shape = ", key1.shape)
print("value_shape = ", value1.shape)
print()

After reshaping... 

query_shape =  (4, 4, 2)
key_shape =  (2, 4, 2)
value_shape =  (2, 4, 2)



### 4. Obtaining the rotary embeddings 

#### 4.1 Pre-computing the sin and cos values

In [20]:
base = 10000
max_seq_len = 2048
dim = head_dim


inv_freq = 1.0 / (base ** (np.arange(0, dim, 2, dtype=np.float32) / dim))
t = np.arange(max_seq_len, dtype=np.float32)
freqs = np.outer(t, inv_freq)
# emb = np.concatenate((np.cos(freqs), np.sin(freqs)), axis=-1)
emb = np.concatenate((freqs,freqs), axis=-1)

cos, sin =  np.cos(emb[:seq_len]), np.sin(emb[:seq_len])

In [21]:
cos, sin

(array([[ 1.       ,  1.       ],
        [ 0.5403023,  0.5403023],
        [-0.4161468, -0.4161468],
        [-0.9899925, -0.9899925]], dtype=float32),
 array([[0.       , 0.       ],
        [0.841471 , 0.841471 ],
        [0.9092974, 0.9092974],
        [0.14112  , 0.14112  ]], dtype=float32))

#### 4.2 Applying the rotations on the Q and K matrices 

In [22]:
# Q matrix rotation

unsqueeze_dim = 0

cos_exp = np.expand_dims(cos, axis=unsqueeze_dim)
sin_exp = np.expand_dims(sin, axis=unsqueeze_dim)


# Half rotation 
q1 = query1[..., :query1.shape[-1] // 2]
q2 = query1[..., query1.shape[-1] // 2:]
q_half_rot = np.concatenate((-q2, q1), axis=-1)


query_rotated = query1*cos_exp + q_half_rot*sin_exp


In [23]:
# K matrix rotation

unskueeze_dim = 0

cos_exp = np.expand_dims(cos, axis=unskueeze_dim)
sin_exp = np.expand_dims(sin, axis=unskueeze_dim)


# Half rotation 
k1 = key1[..., :key1.shape[-1] // 2]
k2 = key1[..., key1.shape[-1] // 2:]
key_half_rot = np.concatenate((-k2, k1), axis=-1)


key_rotated = key1*cos_exp + key_half_rot*sin_exp


In [24]:
query_rotated, key_rotated

(array([[[ 5.06770015e-02,  1.44297732e-02],
         [-5.54562919e-02,  2.27679722e-02],
         [ 2.70013865e-02, -6.95406124e-02],
         [ 1.83803532e-02,  1.87952388e-02]],
 
        [[ 7.17896447e-02, -7.91766867e-03],
         [-1.08548865e-01, -2.31713057e-06],
         [ 1.14325449e-01,  7.97709599e-02],
         [ 3.04648615e-02, -4.91163544e-02]],
 
        [[-5.77881150e-02,  1.41496789e-02],
         [ 7.57668763e-02, -9.93501395e-03],
         [-1.11514568e-01,  4.00876254e-02],
         [ 5.00021204e-02,  7.63263926e-02]],
 
        [[-1.12907983e-01,  6.66608885e-02],
         [ 2.33301278e-02, -1.28501048e-02],
         [ 8.16239566e-02,  1.43432245e-01],
         [-6.05919510e-02, -3.01129147e-02]]], dtype=float32),
 array([[[-0.07981046, -0.02992112],
         [ 0.01709526,  0.0694733 ],
         [ 0.0238962 ,  0.00812519],
         [ 0.04578594,  0.0155874 ]],
 
        [[-0.0730254 , -0.07158848],
         [-0.00553141,  0.1087376 ],
         [-0.05226682, -0.06

In [25]:
value1

array([[[ 0.00158199, -0.06093168],
        [ 0.02432493,  0.04214872],
        [-0.05153289,  0.02271163],
        [ 0.01803326,  0.00495319]],

       [[-0.03399701, -0.04116755],
        [-0.01122927, -0.00305679],
        [-0.0062439 ,  0.0635836 ],
        [ 0.00954244, -0.03041819]]], dtype=float32)

## 5. Masked self attention
### (Grouped Query Attention)

#### 5.1 Repating the K and V values for the GQA

In [26]:
# Repeating the Value vector 

num_value_value_heads, seq_len, head_dim = value1.shape[0], value1.shape[1], value1.shape[2]

if n_rep > 1:

    value1 = np.broadcast_to(value1[:, np.newaxis, :, :], (num_value_value_heads, n_rep, seq_len, head_dim))
    value1 =  value1.reshape(num_value_value_heads * n_rep, seq_len, head_dim)

In [27]:
value1

array([[[ 0.00158199, -0.06093168],
        [ 0.02432493,  0.04214872],
        [-0.05153289,  0.02271163],
        [ 0.01803326,  0.00495319]],

       [[ 0.00158199, -0.06093168],
        [ 0.02432493,  0.04214872],
        [-0.05153289,  0.02271163],
        [ 0.01803326,  0.00495319]],

       [[-0.03399701, -0.04116755],
        [-0.01122927, -0.00305679],
        [-0.0062439 ,  0.0635836 ],
        [ 0.00954244, -0.03041819]],

       [[-0.03399701, -0.04116755],
        [-0.01122927, -0.00305679],
        [-0.0062439 ,  0.0635836 ],
        [ 0.00954244, -0.03041819]]], dtype=float32)

In [28]:
# Repeating the Key vector 

num_key_value_heads, seq_len, head_dim = key_rotated.shape[0], key_rotated.shape[1], key_rotated.shape[2]

if n_rep > 1:

    key_rotated = np.broadcast_to(key_rotated[:, np.newaxis, :, :], (num_key_value_heads, n_rep, seq_len, head_dim))
    key_rotated =  key_rotated.reshape(num_key_value_heads * n_rep, seq_len, head_dim)

In [29]:
key_rotated

array([[[-0.07981046, -0.02992112],
        [ 0.01709526,  0.0694733 ],
        [ 0.0238962 ,  0.00812519],
        [ 0.04578594,  0.0155874 ]],

       [[-0.07981046, -0.02992112],
        [ 0.01709526,  0.0694733 ],
        [ 0.0238962 ,  0.00812519],
        [ 0.04578594,  0.0155874 ]],

       [[-0.0730254 , -0.07158848],
        [-0.00553141,  0.1087376 ],
        [-0.05226682, -0.06412527],
        [ 0.00297071,  0.03554589]],

       [[-0.0730254 , -0.07158848],
        [-0.00553141,  0.1087376 ],
        [-0.05226682, -0.06412527],
        [ 0.00297071,  0.03554589]]], dtype=float32)

In [30]:
query_rotated.shape, key_rotated.shape, value1.shape

((4, 4, 2), (4, 4, 2), (4, 4, 2))

In [31]:
# The attention mask will be created on the go in the self-attention computation

is_causal = True
attn_mask = None 

In [32]:
L, S = query.shape[-2], key.shape[-2]

scale_factor = 1 / np.sqrt(query.shape[-1])
attn_bias = np.zeros((L, S), dtype=query.dtype)

if is_causal:
    assert attn_mask is None
    temp_mask = np.tril(np.ones((L, S), dtype=np.bool_), k=0)
    attn_bias[~temp_mask] = -np.inf
    
    print("Attention bias = ")
    print(attn_bias)

if attn_mask is not None:
    if attn_mask.dtype == np.bool_:
        attn_bias[~attn_mask] = -np.inf
    else:
        attn_bias += attn_mask

# (num_heads, tgt_len, head_dim) @ (num_heads, head_dim, tgt_len) -> (num_heads, tgt_len, tgt_len) 
# attn_weight = np.matmul(query, np.transpose(key, axes=(-2, -1))) * scale_factor
# attn_weight += attn_bias

# # (num_heads, tgt_len, tgt_len) 
# attn_weight = np.exp(attn_weight - np.max(attn_weight, axis=-1, keepdims=True))
# attn_weight /= np.sum(attn_weight, axis=-1, keepdims=True)

# # (num_heads, tgt_len, tgt_len) @ (num_heads, tgt_len, head_dim) -> (num_heads, tgt_len, head_dim) 
# attn_output = np.matmul(attn_weight, value)

# print("ATTEN OUTPUT = ", attn_output)



Attention bias = 
[[  0. -inf -inf -inf]
 [  0.   0. -inf -inf]
 [  0.   0.   0. -inf]
 [  0.   0.   0.   0.]]


In [33]:
key_rotated_T = np.transpose(key_rotated, axes=(0, 2, 1))

attn_weight = query_rotated @ key_rotated_T * scale_factor
attn_weight += attn_bias

exp_attn_weight = np.exp(attn_weight)
sum_exp_attn_weight = np.sum(exp_attn_weight, axis=-1, keepdims=True)
softmax_attn_weight = exp_attn_weight / sum_exp_attn_weight

print("SoftMax (Scaled Dot Product of Q_enc and K_enc) = ")
softmax_attn_weight

SoftMax (Scaled Dot Product of Q_enc and K_enc) = 


array([[[1.        , 0.        , 0.        , 0.        ],
        [0.50027496, 0.499725  , 0.        , 0.        ],
        [0.33349597, 0.33298987, 0.33351418, 0.        ],
        [0.24979158, 0.25011408, 0.2500232 , 0.25007114]],

       [[1.        , 0.        , 0.        , 0.        ],
        [0.5009298 , 0.49907026, 0.        , 0.        ],
        [0.33200294, 0.33424157, 0.33375552, 0.        ],
        [0.24997894, 0.24980846, 0.25009304, 0.2501196 ]],

       [[1.        , 0.        , 0.        , 0.        ],
        [0.49970636, 0.5002937 , 0.        , 0.        ],
        [0.3334242 , 0.33338913, 0.3331867 , 0.        ],
        [0.24932125, 0.25083655, 0.249463  , 0.25037923]],

       [[1.        , 0.        , 0.        , 0.        ],
        [0.5000656 , 0.49993438, 0.        , 0.        ],
        [0.33199075, 0.3356938 , 0.33231544, 0.        ],
        [0.25041625, 0.24957483, 0.25028503, 0.2497239 ]]], dtype=float32)

In [34]:
attn_output = softmax_attn_weight @ value1

In [35]:
attn_output

array([[[ 0.00158199, -0.06093168],
        [ 0.01294721, -0.00941982],
        [-0.00855941,  0.00128928],
        [-0.00189564,  0.00223885]],

       [[ 0.00158199, -0.06093168],
        [ 0.01293232, -0.00948732],
        [-0.00854376,  0.00143849],
        [-0.00190551,  0.00221638]],

       [[-0.03399701, -0.04116755],
        [-0.02260645, -0.02210098],
        [-0.01715953,  0.00643985],
        [-0.01046128, -0.00278502]],

       [[-0.03399701, -0.04116755],
        [-0.02261463, -0.02211467],
        [-0.01713123,  0.00643642],
        [-0.01049573, -0.00275404]]], dtype=float32)

### 6. Post attention 

In [36]:
Wo = state_dict["layers.0.self_attn.o_proj.weight"].numpy()

In [37]:
self_attn_op = np.transpose(attn_output, (1, 0, 2)).copy()
self_attn_op = self_attn_op.reshape(q_len, embed_dim)

sa_output = np.matmul(self_attn_op, Wo.T)


In [38]:
sa_output, sa_output.shape

(array([[ 7.27693457e-03, -2.67719920e-03,  5.26726572e-03,
         -8.76395352e-05,  5.11264989e-05,  2.15278356e-03,
         -5.84058836e-03, -1.33445684e-03],
        [ 3.02217621e-03, -1.56905013e-03,  2.13498180e-03,
          1.01973419e-03,  4.92255320e-04,  6.51503433e-05,
         -3.47597990e-03, -3.22617794e-04],
        [-1.20504480e-03,  3.64525302e-04,  3.06627422e-04,
         -1.15173265e-04, -1.50404885e-04,  3.79330217e-04,
         -2.51639518e-04, -1.32684654e-03],
        [ 2.05713459e-05, -1.64505080e-04,  4.00623016e-04,
          1.57101327e-04,  6.11600190e-05,  3.17409467e-05,
         -6.13048614e-04, -4.55551519e-04]], dtype=float32),
 (4, 8))

In [39]:
# Residual connection

hidden_states = residual + sa_output

hidden_states


array([[-0.01855226,  0.01834289, -0.04304126, -0.0079988 , -0.01634612,
        -0.01183642, -0.0221957 , -0.0019268 ],
       [ 0.01044423, -0.04154749,  0.03251651, -0.01413738,  0.01578901,
         0.02892418,  0.00022154,  0.02463173],
       [ 0.02787066,  0.00399948, -0.01002509, -0.01450835,  0.00559201,
        -0.00265816, -0.01615272, -0.04222466],
       [ 0.02146314, -0.01154592,  0.000114  ,  0.04154483,  0.00130236,
         0.0080259 , -0.02327467,  0.00996811]])

In [40]:
residual = hidden_states

#### 7. LayerNorm 2 

In [41]:
variance_epsilon = 1e-05

wt = state_dict["layers.0.post_attention_layernorm.weight"].numpy()

dtype = hidden_states.dtype
hidden_states = hidden_states.astype(np.float32)
variance = np.mean(hidden_states**2, axis=-1, keepdims=True)
hidden_states = hidden_states * (1/np.sqrt(variance + variance_epsilon))

hidden_state = wt * hidden_states

In [42]:
hidden_state

array([[-0.87689877,  0.86700267, -2.0344064 , -0.37807474, -0.7726225 ,
        -0.55946547, -1.0491114 , -0.09107281],
       [ 0.4235199 , -1.6847762 ,  1.3185644 , -0.5732795 ,  0.64025396,
         1.1728932 ,  0.00898345,  0.9988315 ],
       [ 1.3793948 ,  0.19794495, -0.49616924, -0.7180578 ,  0.2767637 ,
        -0.1315597 , -0.7994422 , -2.0898135 ],
       [ 1.0890743 , -0.58585846,  0.0057843 ,  2.1080515 ,  0.06608368,
         0.407247  , -1.1809942 ,  0.50579804]], dtype=float32)

### 8. MLP layer of llama

In [43]:
W_down_proj = state_dict["layers.0.mlp.down_proj.weight"].numpy()

W_up_proj = state_dict["layers.0.mlp.up_proj.weight"].numpy()

W_gate_proj = state_dict["layers.0.mlp.gate_proj.weight"].numpy()

In [44]:
up_proj = np.matmul(hidden_state, W_up_proj.T)
gate_proj = np.matmul(hidden_state, W_gate_proj.T)

print("UP PROJ = ", up_proj)
print()

print("GATE PROJ = ", gate_proj)
print()

temp_proj = up_proj * gate_proj


# SilU 
temp_proj = temp_proj / (1 + np.exp(-temp_proj))

print("ACT = ", temp_proj)

down_proj = np.matmul(temp_proj, W_down_proj.T)


UP PROJ =  [[-0.07960668 -0.04966336 -0.11909581 ... -0.02586847  0.02103351
  -0.01271885]
 [ 0.05847102  0.05472086  0.08879388 ...  0.00213997 -0.0127653
   0.02411557]
 [-0.00236438 -0.07412671 -0.0655765  ... -0.10071106 -0.09066673
  -0.05119118]
 [ 0.00137428  0.05476112  0.06004668 ... -0.02531331 -0.00841016
   0.05105576]]

GATE PROJ =  [[ 0.05269722  0.08074224  0.10290758 ... -0.03795651 -0.01921894
   0.00335514]
 [-0.06081866 -0.04648931 -0.09590532 ... -0.04052179 -0.04078406
   0.01393612]
 [ 0.0068357   0.09731894 -0.02760312 ...  0.04183926  0.01476397
   0.03711531]
 [-0.07632128 -0.08056262 -0.03327958 ...  0.03036338  0.02704049
   0.0447953 ]]

ACT =  [[-2.0931258e-03 -2.0009456e-03 -6.0903803e-03 ...  4.9117941e-04
  -2.0208010e-04 -2.1336316e-05]
 [-1.7749032e-03 -1.2703494e-03 -4.2397734e-03 ... -4.3355820e-05
   2.6037823e-04  1.6806697e-04]
 [-8.0810387e-06 -3.5939561e-03  9.0587704e-04 ... -2.1023995e-03
  -6.6885230e-04 -9.4908575e-04]
 [-5.2440584e-05 -2.2

In [46]:
hidden_state = down_proj
down_proj

array([[-8.4653869e-04, -9.3271758e-04, -7.8715227e-04,  1.3892719e-03,
        -3.4011441e-04,  1.0799251e-03,  4.2393636e-03,  1.2252626e-03],
       [ 2.5297564e-03, -1.2564536e-03, -3.1192109e-03, -1.5179769e-03,
        -1.7135046e-04,  3.5857344e-03,  2.0369976e-03,  4.7027890e-05],
       [-5.8930280e-04, -4.1200305e-04, -4.8883278e-03,  1.3618954e-03,
        -1.8036359e-03,  4.2749709e-03,  1.1712741e-03, -5.6474784e-04],
       [-3.1879538e-04,  8.3013141e-04,  1.9765906e-03,  1.2665620e-03,
         1.4343827e-03, -1.0643373e-03,  8.1461773e-04, -1.9027097e-03]],
      dtype=float32)

In [47]:
# Residual connection
hidden_state = hidden_state + residual

In [48]:
hidden_state

array([[-0.01939879,  0.01741017, -0.04382841, -0.00660953, -0.01668623,
        -0.0107565 , -0.01795634, -0.00070153],
       [ 0.01297399, -0.04280394,  0.0293973 , -0.01565536,  0.01561766,
         0.03250992,  0.00225853,  0.02467875],
       [ 0.02728135,  0.00358747, -0.01491342, -0.01314645,  0.00378837,
         0.00161681, -0.01498144, -0.0427894 ],
       [ 0.02114435, -0.01071579,  0.00209059,  0.04281139,  0.00273674,
         0.00696156, -0.02246005,  0.0080654 ]])